In [ ]:
# uncomment if running in vs code
# %cd ..
# %cd ..
# %cd ..

## Kernel: run_compound
Run the 64 simulations for the current approach and compare the risk curves with TGP on test dataset

In [3]:
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata

import Notebooks.Scripts.normalization as normalizer
import Notebooks.Scripts.sampling_utils as sam_util
from Notebooks.Scripts import selector_mda

In [ ]:
df_two = pd.read_csv('fitted_stats/2d_sims.csv')
df_two.head()

In [ ]:
stoc_set = df_two
minmax_scaler = normalizer.scaler(stoc_set)
df_minmax_scaled = normalizer.normalize_dataset(stoc_set, minmax_scaler)

corner_combinations = list(itertools.product([0, 1], repeat=df_two.shape[1]))
df_grid = pd.DataFrame(corner_combinations, columns=df_minmax_scaled.columns)

seed = df_minmax_scaled['S Mag'].argmax()

mda_runs = 8**stoc_set.shape[1] - 2**stoc_set.shape[1]

maxmin_class = selector_mda.MaxMin()
lst_ind = maxmin_class.select_from_cluster(df_minmax_scaled.values, mda_runs, seed)
subset = df_minmax_scaled.iloc[lst_ind].copy(deep = True)

scaled_sims = pd.concat((df_grid, subset)).reset_index(drop = True)

denorm_sims = normalizer.denormalize_dataset(scaled_sims, minmax_scaler).values
denorm_sims = pd.DataFrame(denorm_sims, columns = stoc_set.columns)
ax = df_two.plot(kind = 'scatter', x = 'S Mag [m]', y = 'P Mag [mm/hr]', label = 'Available')
denorm_sims.plot(kind = 'scatter', x = 'S Mag [m]', y = 'P Mag [mm/hr]', ax = ax, c= 'k', label = 'MDA')
denorm_sims.iloc[:2**df_two.shape[1]].plot(kind = 'scatter', x = 'S Mag [m]', y = 'P Mag [mm/hr]', ax = ax, c= 'r', label = 'Corners')

In [ ]:
target, times, paths = sam_util.run_MDA('MDA_2d', denorm_sims, len(stoc_set),
                                        output = ['Total'], bonus = None, plot = False, artificial_tide=False)

In [8]:
target = pd.read_csv('Models/MDA_2d/damages.csv', index_col = 0)

In [ ]:
sampled_events = denorm_sims.copy(deep = True)
sampled_events['Total'] = target[:len(denorm_sims)]
sampled_events

In [10]:
grid_x, grid_y = np.meshgrid(np.linspace(df_two['S Mag [m]'].min(), df_two['S Mag [m]'].max(), 1000),
                             np.linspace(df_two['P Mag [mm/hr]'].min(), df_two['P Mag [mm/hr]'].max(), 1000), indexing='ij')

In [ ]:
grid_target = griddata(sampled_events.iloc[:, :2], sampled_events.iloc[:, 2], (grid_x, grid_y), method='linear')
interp_target = griddata(sampled_events.iloc[:, :2], sampled_events.iloc[:, 2], df_two.values, method='linear')

contour_levels = np.linspace(target.min().item(), target.max().item(), 11)
plt.figure(figsize = (10, 7))
plt.contourf(grid_x,
             grid_y,
             grid_target,
             cmap='viridis',
             levels = contour_levels)
plt.colorbar(label = 'Total Economic Damages [USD]').ax.yaxis.label.set_size(12)
plt.scatter(x = df_two['S Mag [m]'], y = df_two['P Mag [mm/hr]'], c = 'k', label = 'Available', s = 2)
plt.scatter(x = denorm_sims['S Mag [m]'], y = denorm_sims['P Mag [mm/hr]'], c= 'r', label = 'MDA', s = 2)
plt.scatter(x = denorm_sims.iloc[:2**df_two.shape[1]]['S Mag [m]'], y = denorm_sims.iloc[:2**df_two.shape[1]]['P Mag [mm/hr]'], c= 'g', label = 'Corners', s=20)
plt.xlabel('S Mag [m]', fontsize = 12)
plt.ylabel('P Mag [mm/hr]', fontsize = 12)
plt.legend(fontsize = 12)

In [ ]:
interp = df_two.copy(deep = True)
interp['Total [USD]'] = interp_target
interp